In [ ]:
## Parameters to be passed in from pipeline
minSku = 'F2' # min sku size we can scale down to
maxSku = 'F128' # max sku size we can scale up to
utilizationTolerance = 90 # Percentage of CU used to stay under. Its to say, we'll scale to a sku where we're under the utilizationTolerance so consumed CU < F SKU * utilizationTolerance
capacityName = '' #capacity name to be monitored. Might need to also require region or capacity Id since capacityName and region are what makes a capacity name unique
subscriptionId = '' # Azure subscription that the capacity is in
metricsAppWorkspaceName = 'WS_FabricCapacityMetrics' # Name of the workspace that the Capacity Metric App semantic model is in. Could default to the default workspace
metricsAppModelName = 'Fabric Capacity Metrics' # name of the Capacity Metric App semantic model. Could default to the default app name

In [ ]:
keyVaultEndpoint = ''

tenantId = mssparkutils.credentials.getSecret(keyVaultEndpoint, 'secretName_tenantId')
clientId = mssparkutils.credentials.getSecret(keyVaultEndpoint, 'secretName_clientId')
secret = mssparkutils.credentials.getSecret(keyVaultEndpoint, 'secretName_clientSecret')

##### Token Audiences and tokens

In [ ]:
from azure.identity import ClientSecretCredential

api_pbi = 'https://analysis.windows.net/powerbi/api/.default'
api_azuremgmt = 'https://management.core.windows.net/.default'

auth = ClientSecretCredential(tenant_id=tenantId, client_id=clientId, client_secret=secret)
header_pbi = {'Authorization': f'Bearer {auth.get_token(api_pbi).token}', 'Content-type': 'application/json'}
header_azuremgmt = {'Authorization': f'Bearer {auth.get_token(api_azuremgmt).token}', 'Content-type': 'application/json'}

##### Refresh only the imported tables required

In [ ]:
import requests, json, time

response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header_pbi)

workspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == metricsAppWorkspaceName][0]
print(f'{workspaceId = }')

response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets", headers=header_pbi)

datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == metricsAppModelName][0]
print(f'{datasetId = }')

# https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#post-refreshes
tableList = [{"table": "Capacities"}
            ,{"table": "TimePoints"}
            ,{"table": "Items"}
            ]
body = {"objects": tableList} # Need to ask Pat what tables are import and what are direct query
response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/refreshes", headers=header_pbi, data=json.dumps(body))

refreshId = response.headers.get('RequestId')
print(f'{refreshId = }')

# Check the status of the refresh attempting 12 times ~60 seconds
for attempt in range(12): 
    # https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#get-refreshes
    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/refreshes?$top=1", headers=header_pbi)
    if response.json().get('value')[0].get('status') != 'Unknown':
        print(f'Refresh Complete')
        break
    else:
        print(f'Refreshing tables ...')
        time.sleep(5)

##### Create a dictionary for each Fabric capacity sku and the total amount of capacity unit seconds over a 24 hour period

In [ ]:
skuDict = {f'F{2**_}': (2**_)*(60*60*24) for _ in range(1,12)}
print(f'{skuDict = }')

##### Get the current sku of the capacity, the amount of capacity seconds per 24 hours, and capacity id

In [ ]:
import requests

response = requests.get("https://api.fabric.microsoft.com/v1/capacities", headers=header_pbi)

currentSku = [capacity.get('sku') for capacity in response.json().get('value') if capacity.get('displayName') == capacityName][0]
print(f'{currentSku = }')
currentSkuCUTotalPerDay = skuDict[currentSku]
print(f'{currentSkuCUTotalPerDay = }')
capacityId = [capacity.get('id') for capacity in response.json().get('value') if capacity.get('displayName') == capacityName][0].upper()
print(f'{capacityId = }')

##### Query the Fabric Capacity Metrics App model to get the total consumed capacity seconds for the last 24 hours

In [ ]:
import requests, json, math

body = {
  "queries": [
    {
      "query": f"""
        DEFINE
            MPARAMETER 'CapacityID' = "{capacityId}"

            VAR __DS0FilterTable2 = TREATAS({{"{capacityId}"}}, 'Capacities'[capacityId])
            
            VAR __DS0FilterTable3 = 
                      FILTER(
                        KEEPFILTERS(VALUES('TimePoints'[TimePoint])),
                        'TimePoints'[TimePoint] >= NOW() - 1
                      )
                      
            VAR __DS0Core = 
              SELECTCOLUMNS(
                SUMMARIZECOLUMNS(
                'Capacities'[Capacity Name],
                Items[Billable type],
                  __DS0FilterTable2,
                  __DS0FilterTable3,
                  "SumInteractive", SUM('CUDetail'[Interactive]),
                  "SumBackground", SUM('CUDetail'[Background]),
                  "SumCUs", SUM('CUDetail'[CUs])
                ),
                "CapacityName", 'Capacities'[Capacity Name],
                "BillType", Items[Billable type],
                "SumInteractive", [SumInteractive],
                "SumBackground", [SumBackground],
                "SumCUs", [SumCUs]
              )

          EVALUATE
            __DS0Core
    """
    }
  ]
}

response = requests.post(f'https://api.powerbi.com/v1.0/myorg/datasets/{datasetId}/executeQueries', headers=header_pbi, json=body )

totalConsumedCULast24HoursInteractive = 0
totalConsumedCULast24HoursBackground = 0
for results in response.json().get('results'):
    for table in results.get('tables'):
        for row in table.get('rows'):
          totalConsumedCULast24HoursInteractive += row.get('[SumInteractive]')
          totalConsumedCULast24HoursBackground += row.get('[SumBackground]')

totalConsumedCULast24Hours = math.ceil(totalConsumedCULast24HoursInteractive + totalConsumedCULast24HoursBackground)
print(f'{totalConsumedCULast24Hours = }')

##### Apply logic to determine if the capacity should be scaled and if it needs to scale, what capacity should it scale to based on the current capacity, consumption over the last 24 hours, and the defined utilization tolerance to be within the defined min/max SKU

In [ ]:
import math

utilizationTolerancePercentage = utilizationTolerance/100

print(f'{totalConsumedCULast24Hours = }')
print(f'{utilizationTolerancePercentage = }')
print(f'{currentSkuCUTotalPerDay = }')

skuNeeded = [(sku, cu, totalConsumedCULast24Hours, math.ceil(cu*utilizationTolerancePercentage)) for sku, cu in skuDict.items() if math.ceil(cu*utilizationTolerancePercentage) >= totalConsumedCULast24Hours][0]
print(f'{skuNeeded = }')

scaleSku = ''
if int(skuNeeded[0].replace('F', '')) < int(minSku.replace('F', '')):
    scaleSku = minSku
elif int(skuNeeded[0].replace('F', '')) > int(maxSku.replace('F', '')):
    scaleSku = maxSku
else:
    scaleSku = skuNeeded[0]

print(f'{scaleSku = }')

##### Perform the scaling operation within Azure

In [ ]:
import requests, json

# One last validation to check if the sku to scale to is different than the current sku
if scaleSku != currentSku:
    print(f'\nScaling from {currentSku} to {scaleSku}')

    response = requests.get(f'https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.Fabric/capacities?api-version=2022-07-01-preview', headers=header_azuremgmt)
    responseList = response.json().get('value')
    resourceGroupName = [resource.get('id') for resource in responseList if resource.get('name') == capacityName][0].split("resourceGroups/")[-1].split("/")[0]

    url = f'https://management.azure.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/providers/Microsoft.Fabric/capacities/{capacityName}?api-version=2022-07-01-preview'
    body = {"sku": {"name": f"{scaleSku}", "tier": "Fabric"}}
            
    response = requests.patch(url, headers=header_azuremgmt, data=json.dumps(body))
    print(response, response.text)

else:
    print(f'The capacity {currentSku} is already under the utilization tolerance setting of {utilizationTolerance}%.')